In [2]:
import pandas as pd
import numpy as np
import math
import os
import pickle
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [3]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [4]:
# naive basin size of a single genotype within a fitness landscape
def basin_size(landscape, genotype, num=10, **kwargs):
    size = 0
    if isinstance(landscape, list):
        name = ' + '.join([ls.name for ls in landscape])
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    param = dict(kwargs)
    for seed in range(16):
        param['seed'] = seed
        for i in range(num):
            results = simulate(landscape, **param)
            if results['T_f'] != -1 and results['actual_path'][-1] == genotype:
                size += 1
    return size / num

In [5]:
def basin_size2(landscape, genotype, num=10, **kwargs):
    sizes = []
    if isinstance(landscape, list):
        name = ' + '.join([ls.name for ls in landscape])
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    param = dict(kwargs)
    for i in range(num):
        size = 0
        for seed in range(16):
            param['seed'] = seed
            results = simulate(landscape, **param)
            if results['T_f'] != -1 and results['actual_path'][-1] == genotype:
                size += 1
        sizes.append(size)
    return sizes

In [6]:
# get basin sizes for each genotype in a fitness landscape
def basin_sizes(landscape, k=9, **kwargs):
    sizes = []
    for i in range(16):
        gen = format(i, '04b')
        #bs = basin_size(landscape, gen, carrying_cap=10**k, prob_mutation=10**(-(k-1)), **kwargs)
        bs = basin_size(landscape, gen, carrying_cap=10**k, **kwargs)
        print('.', end='')
        sizes.append(bs)
    print()
    return sizes

In [6]:
# generate and serialize some collections of basin sizes
with open('basin_sizes.pickle', 'a+b') as f:
    f.seek(0)
    try:
        data = {}
        data.update(pickle.load(f))
    except:
        data = {}
        
# basin sizes for all drugs in the 2nd dataset
if 'dataset2' not in data:
    data['dataset2'] = {}
for name, ls in dataset2.iterrows():
    if name not in data['dataset2']:
        data['dataset2'][name] = basin_sizes(ls)

# basin sizes for all drugs in the 2nd dataset at k=10^6
if 'dataset2_6' not in data:
    data['dataset2_6'] = {}
for name, ls in dataset2.iterrows():
    if name not in data['dataset2_6']:
        data['dataset2_6'][name] = basin_sizes(ls, k=6)
        
# basin sizes for the pairs at f=200
if 'pairs' not in data:
    data['pairs'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs']:
        data['pairs'][i] = basin_sizes(ls, k=9, frequency=200)

# basin sizes for the pairs at f=200 / k=10^6
if 'pairs_6' not in data:
    data['pairs_6'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_6']:
        data['pairs_6'][i] = basin_sizes(ls, k=6, frequency=200)

# basin sizes for the pairs at f=100
if 'pairs_100' not in data:
    data['pairs_100'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_100']:
        data['pairs_100'][i] = basin_sizes(ls, k=9, frequency=100)

# basin sizes for the pairs at f=25
if 'pairs_25' not in data:
    data['pairs_25'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_25']:
        data['pairs_25'][i] = basin_sizes(ls, k=9, frequency=25)

# basin sizes for the pairs at f=75
if 'pairs_75' not in data:
    data['pairs_75'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_75']:
        data['pairs_75'][i] = basin_sizes(ls, k=9, frequency=75)

# basin sizes for the pairs at f=50
if 'pairs_50' not in data:
    data['pairs_50'] = {}
for i, ls in enumerate(pairs):
    if i not in data['pairs_50']:
        data['pairs_50'][i] = basin_sizes(ls, k=9, frequency=50)

with open('basin_sizes.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

bs = data

# Basin size

For a given genotype, the basin size is the number of other genotypes it can be reached from.

Reachability--One genotype is reachable from another if a simulation run on a starting population consisting entirely of the first genotype results in fixation of the second genotype after 1200 timesteps. (We could define this in other ways; for example, the abundance of the second genotype reaching a certain threshold at some point during the simulation could mean it is reachable.)

Since simulations can have different outcomes, the code above essentially takes an average of the basin size for each genotype after running 10 simulations.

## Basin size for static landscapes

Non-zero basin sizes appear to correspond to the local and global optima that we [found earlier](https://nbviewer.jupyter.org/url/jops.bol.ucla.edu/18-11-01/Oct-31-many-sims-w-switching.ipynb#K=10^9).

In [72]:
data=[]
for name in dataset2.index:
    data.append(go.Scatter(
        y=bs['dataset2'][name],
        name=name
    ))
fig = tools.make_subplots(rows=len(data), cols=1, print_grid=False)
for i, d in enumerate(data, 1):
    fig.append_trace(d, i, 1)
    fig['layout'][f'xaxis{i}'].update(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        range=[-0.1,15.1],
        zeroline=False
    )
    fig['layout'][f'yaxis{i}'].update(
        range=[-1,17],
        tickvals=[0,8,16],
        title=d.name,
        showticklabels=True
    )
    
fig['layout'].update(
    height=1050,
    width=850,
    showlegend=False,
    title='Basin sizes for static landscapes',
    margin=go.layout.Margin(r=20,l=50,t=50, b=30)
)
iplot(fig, show_link=False)
pio.write_image(fig, 'report/figx2.png', scale=3)

In [35]:
plot_simulation(simulate([dataset2.loc['FEP'].tolist(), dataset2.loc['CEC'].tolist()], frequency=100))

In [24]:
CEC = dataset2.loc['CEC'].tolist()
FEP = dataset2.loc['FEP'].tolist()
plot_simulation(simulate([CEC, FEP, CEC], durations=[200,100,100]))

In [59]:
ZOX = dataset2.loc['ZOX'].tolist()
TZP = dataset2.loc['TZP'].tolist()
plot_simulation(simulate([ZOX, TZP, ZOX], durations=[600, 60, 600], seed='1010'))

In [71]:
data = [
    go.Scatter(y=bs['dataset2'][dataset2.index[0]], name='10^9'),
    go.Scatter(y=bs['dataset2_6'][dataset2.index[0]], name='10^6')
]
buttons = []
for name in dataset2.index:
    buttons.append(dict(
        args=[{'y': [bs['dataset2'][name], bs['dataset2_6'][name]]}],
        label=name,
        method='restyle'
    ))

updatemenus=list([
    dict(
        buttons=buttons,
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

layout = go.Layout(
    title='Comparison of basin sizes at two carrying capacities',
    xaxis=dict(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        title='Genotype',
        zeroline=False
    ),
    yaxis=dict(
        title='Basin size',
        range=[-1,17]
    ),
    updatemenus=updatemenus
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

## Basin size for dynamic landscapes

In [27]:
layout = go.Layout(
    xaxis=dict(
        ticktext=[format(i, '04b') for i in range(16)],
        tickvals=list(range(16)),
        title='Genotype',
        zeroline=False
    ),
    yaxis=dict(
        title='Basin size',
        range=[-1,17]
    )
)

for i, pair in enumerate(pairs):
    name = ' + '.join([ls.name for ls in pair])
    data = [
        go.Scatter(y=bs['pairs'][i], name='f=200'),
        go.Scatter(y=bs['pairs_50'][i], name='f=50', line=dict(dash='dash'))
    ]
    layout.update(title=name)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [22]:
frequencies=[200,100,75,50,25,20,15,10,5,1]
data=[go.Scatter(
    y=[basin_size(p, opt, frequency=f) for f in frequencies],
    x=frequencies,
    name=' + '.join([ls.name for ls in p])
) for p,opt in zip(pairs, ['1111', '1101', '0111'])]

In [36]:
layout=go.Layout(
    xaxis=dict(
        range=[210,-10],
        tickvals=[200,150,100,75,50,25,20,15,10,5,1],
        zeroline=False,
        title='Switching frequency'
    ),
    yaxis=dict(
        title='Basin size'
    ),
    title='Basin size of the optimal genotype at different frequencies',
    width=1000
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)
pio.write_image(fig, 'report/figx.png', scale=3)

In [47]:
def many_simulations(landscape, param={}, num=100):
    success_count = 0
    greedy_path = ''
    paths = {}
    T_f_sum = 0
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **param)
        if results['T_f'] != -1:
            success_count += 1
            T_f_sum += results['T_f']
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = 0
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path] += 1
        else:
            paths[actual_path] = 1
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
    return {
        'Success rate': success_count / num,
        '# of paths': len(paths),
        'Path frequencies': paths.values(),
        'Greedy path': greedy_path,
        'Greedy rate': paths[greedy_path] / num,
        'Avg time to fixation': T_f_sum / num,
        'Global optimum': global_optimum,
        'Local optima': local_optima
    }

# ordering
column_names =  ['Success rate', '# of paths', 'Path frequencies', 'Greedy path', 
                 'Greedy rate', 'Local optima', 'Global optimum', 'Avg time to fixation']   

def many_landscapes(param={}, df=dataset2):
    data = []
    for name, ls in df.iterrows():
        display('Running simulations on {}...'.format(name))
        landscape = ls.tolist()
        if name == 'ZOX':
            row = many_simulations(landscape, dict(param, timesteps=10000))
        else:
            row = many_simulations(landscape, param)
        row['Name'] = name
        data.append(row)
    clear_output()
    return pd.DataFrame(data).set_index('Name').reindex(column_names, axis='columns')

In [49]:
many_landscapes({'carrying_cap': 10**9, 'timesteps' : 5000})

,Success rate,# of paths,Path frequencies,Greedy path,Greedy rate,Local optima,Global optimum,Avg time to fixation
Name,,,,,,,,
AMP,0.0,1,(100),"0000,0001,0011",1.00,"0011, 0110",1111,0.00
AM,0.0,1,(100),"0000,0010",1.00,0010,1101,0.00
CEC,0.0,1,(100),"0000,0100",1.00,"0100, 1110",0011,0.00
CTX,0.0,1,(100),"0000,0010,0011",1.00,"0011, 0110, 1010",1111,0.00
ZOX,1.0,1,(100),"0000,0010,0011,0111",1.00,1001,0111,3861.83
CXM,0.0,1,(100),"0000,0100",1.00,0100,0111,0.00
CRO,0.0,3,"(61, 22, 17)","0000,0100",0.61,"0011, 0100, 1010",1111,0.00
AMC,0.0,1,(100),"0000,0100",1.00,0100,1101,0.00
CAZ,1.0,2,"(0, 100)","0000,0001,0101",0.00,"0011, 0101",0110,125.63


In [48]:
many_landscapes({'carrying_cap': 10**7, 'timesteps' : 3000})

,Success rate,# of paths,Path frequencies,Greedy path,Greedy rate,Local optima,Global optimum,Avg time to fixation
Name,,,,,,,,
AMP,0.23,5,"(76, 12, 1, 10, 1)","0000,0001,0011",0.76,"0011, 0110",1111,106.72
AM,0.00,1,(100),"0000,0010",1.00,0010,1101,0.00
CEC,0.00,1,(100),"0000,0100",1.00,"0100, 1110",0011,0.00
CTX,0.00,3,"(43, 48, 9)","0000,0010,0011",0.43,"0011, 0110, 1010",1111,0.00
ZOX,1.00,4,"(68, 10, 21, 1)","0000,0010,0011,0111",0.68,1001,0111,2819.87
CXM,0.10,2,"(90, 10)","0000,0100",0.90,0100,0111,20.72
CRO,0.00,3,"(53, 15, 32)","0000,0100",0.53,"0011, 0100, 1010",1111,0.00
AMC,0.04,2,"(96, 4)","0000,0100",0.96,0100,1101,21.56
CAZ,0.50,2,"(50, 50)","0000,0001,0101",0.50,"0011, 0101",0110,78.31


In [33]:
pd.DataFrame({x['Success rate'] for x in [K10, K9, K8, K7, K6, K5]})

Name,AMP,AM,CEC,CTX,ZOX,CXM,CRO,AMC,CAZ,CTT,SAM,CPR,CPD,TZP,FEP
Success rate,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,1.00,0.00,1.00,1.00,1.00,1.00,0.0
Success rate,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,1.00,0.00,1.00,1.00,1.00,1.00,0.0
Success rate,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.51,0.00,1.00,1.00,1.00,1.00,0.0
Success rate,0.19,0.0,0.0,0.0,0.35,0.12,0.0,0.05,0.40,0.05,1.00,0.97,1.00,1.00,0.0
Success rate,0.26,0.0,0.0,0.0,0.38,0.31,0.0,0.16,0.36,0.07,0.27,0.29,0.32,0.87,0.0
Success rate,0.00,0.0,0.0,0.0,0.00,0.01,0.0,0.01,0.02,0.01,0.00,0.00,0.00,0.05,0.0


In [19]:
data=[]
for k, x_offset in zip([9, 7, 5], [-.1, 0, .1]):
    x=[]
    y=[]
    for g in range(15):
        times = [simulate(dataset2.iloc[g].tolist(), carrying_cap=10**k, timesteps=3000)['T_f'] for i in range(100)]
        for t in times:
            if t != -1:
                x.append(g + x_offset)
                y.append(t)
    data.append(go.Scatter(x=x, y=y, name='10^{}'.format(k), mode='markers', marker=dict(size=5)))

In [21]:
layout = go.Layout(
    xaxis=dict(
        range=[-1,15],
        tickvals=list(range(15)),
        ticktext=dataset2.index.tolist(),
        zeroline=False,
        title='Landscape'
    ),
    yaxis=dict(
        range=[0,3100],
        title='Time to fixation'
    ),
    title='Running simulations using three different carrying capacities'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)
pio.write_image(fig, 'report/figc1.png', scale=3)

In [90]:
# figure 1
column_names =  ['Success rate', 'Greedy path', 'Greedy rate', 'Local optima', 'Global optimum', 'Avg time to fixation']   
many_landscapes()

,Success rate,Greedy path,Greedy rate,Local optima,Global optimum,Avg time to fixation
Name,,,,,,
AMP,0.0,"0000,0001,0011",1.00,"0011, 0110",1111,0.00
AM,0.0,"0000,0010",1.00,0010,1101,0.00
CEC,0.0,"0000,0100",1.00,"0100, 1110",0011,0.00
CTX,0.0,"0000,0010,0011",1.00,"0011, 0110, 1010",1111,0.00
ZOX,1.0,"0000,0010,0011,0111",1.00,1001,0111,3854.40
CXM,0.0,"0000,0100",1.00,0100,0111,0.00
CRO,0.0,"0000,0100",0.57,"0011, 0100, 1010",1111,0.00
AMC,0.0,"0000,0100",1.00,0100,1101,0.00
CAZ,1.0,"0000,0001,0101",0.00,"0011, 0101",0110,125.40


In [7]:
def plot_simulation2(results, genotypes_to_plot=[], carrying_capacity=9):
    trace = results['trace']
    if not genotypes_to_plot:
        genotypes_to_plot = list(
            set(results['greedy_path']) | set(results['actual_path'])
        )
        genotypes_to_plot.sort(key=lambda g: np.argmax(trace[g] > 0))
    y_max = carrying_capacity + 1
    timesteps = len(next(iter(trace.values())))
    #           length of an arbitrary entry from dictionary
    index = np.array(range(0, timesteps), np.int64)
    
    # data
    data = []
    for g in genotypes_to_plot:
        data.append(go.Scatter(
            x=index,
            y=trace[g],
            name=g,
            line=dict(width=1)
        ))

    # vertical lines
    for crit in ['T_1', 'T_d', 'T_f']:
        if results[crit] != -1:
            data.append(go.Scatter(
                x=[results[crit], results[crit]],
                y=[0, 10**y_max],
                line=dict(color='black', width=1),
                name = crit,
                mode = 'lines',
                showlegend = False
            ))

    # vertical line annotations
    vlines = []
    for crit, text in zip(['T_1', 'T_d', 'T_f'],
                          ['<i>T</i><sub>1</sub>',
                           '<i>T</i><sub>d</sub>',
                           '<i>T</i><sub>f</sub>']):
        if results[crit] != -1:
            vlines.append(dict(
                x=results[crit],
                y=y_max*.975,
                xref='x',
                yref='y',
                text=text,
                showarrow=False,
                xanchor='left'
            ))
        
    # layout
    layout = go.Layout(
        xaxis=dict(
            title='Timestep'
        ),
        yaxis=dict(
            type='log',
            range=[0,y_max],
            exponentformat='power',
            title='Abundance of each genotype',
            tickfont=dict(size=10)
        ),
        width=600, #600
        height=400, #400
        margin=dict(t=30, b=40, l=50, r=0),
        annotations=vlines
    )
        
    fig = go.Figure(data=data, layout=layout)
    return fig

In [133]:
for n in ['CTX', 'SAM', 'AM', 'AMC', 'ZOX', 'CXM']:
    fig = plot_simulation2(simulate(dataset2.loc[n].tolist()))
    fig.layout.update(title=n)
    pio.write_image(fig, 'report/fig2/{}.png'.format(n), scale=3)

In [134]:
fig = plot_simulation2(simulate([dataset2.loc['AM'].tolist(), dataset2.loc['AMC'].tolist()], frequency=200))
fig.layout.update(title='AM+AMC at f=200')
pio.write_image(fig, 'report/fig3.png', scale=3)

In [8]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    times = [] # T_f's
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
        times.append(results['T_f'])
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if not np.isnan(max_weight):
        dominant_path = df.iloc[df['Weight'].idxmax()]['Pathway']
    else:
        dominant_path = ""
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            'success_rate': num_successful_paths / num_paths,
            'dominant_path': dominant_path,
            'times': times
        }

def box_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    for f in frequencies:
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = go.Box(
            #x = [f for i in range(len(times))],
            y = times,
            boxpoints = False,
            name = 'f={}'.format(f)
        )
        data.append(trace)
        wx.append('f={}'.format(f))
        wy.append(analysis['success_count'] / num)
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Success rate',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        ),
        mode='markers'
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at different frequencies'.format(num, name),
        xaxis = dict(
            title = 'Switching frequency',
            tickvals = ['f={}'.format(f) for f in frequencies]
        ),
        yaxis = dict(
            title = 'Time to fixation',
            range=[0,1400]
        ),
        yaxis2 = dict(
            title = 'Success rate',
            overlaying = 'y',
            side = 'right',
            range=[-0.2,1.2],
            tickvals=[0,0.2,0.4,0.6,0.8,1.0],
            zeroline=False
        ),
        showlegend = False
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [76]:
for p in pairs:
    fig = box_switching(p)
    #iplot(fig, show_link=False)
    pio.write_image(fig, 'report/fig4/{}.png'.format('+'.join([ls.name for ls in p])), scale=3)

In [15]:
for k in [7]:
    fig = box_switching(pairs[0], k=k)
    fig['layout'].update(title='{} at K=10<sup>{}</sup>'.format('+'.join([ls.name for ls in pairs[0]]), k))
    #iplot(fig, show_link=False)
    pio.write_image(fig, 'report/figx3.png', scale=3)